In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

boxes = ['gearbox00', 'gearbox10', 'gearbox20', 'gearbox30', 'gearbox40']
sensors = ['sensor1', 'sensor2', 'sensor3', 'sensor4']

In [10]:
# 读取原始数据，initial_data

initial_data = pd.read_excel('原始数据.xlsx')


In [19]:
X = np.array(initial_data[sensors[:2]])

y = np.array(initial_data[sensors[-2]])
y.resize(len(y),1)

In [28]:
# 归一化处理

X_scaler= MinMaxScaler(feature_range=(-1,1))
y_scaler = MinMaxScaler(feature_range=(-1,1))

X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

In [31]:
# 归一化后放入excel

temp1 = pd.DataFrame(X, columns=sensors[:2])
temp2 = pd.DataFrame(y, columns=[sensors[-2]])

d = pd.concat([temp1,temp2],axis=1)

d.to_csv('归一化后数据.csv')

In [14]:
# 对样本进行转置，只取其中的25000条数据进行训练，剩下4400条进行准确度的计算
sample_in = X[:25000].T
sample_out = y[:25000].T

In [21]:
# 设置bp网络参数
max_epochs = 3000
learn_rate = 0.035
mse_final = 6.5e-4
sample_number = sample_in.shape[1]
input_number = 2
output_number = 1
hidden_unit_number = 8


In [22]:
# 学习训练超参
# w1,b1:输入层到隐藏层
# 8*2
w1 = 0.5 * np.random.rand(hidden_unit_number, input_number) - 0.1
# 8*1
b1 = 0.5 * np.random.rand(hidden_unit_number, 1) - 0.1

# w2,b2:隐藏层到输出层
# 2*8
w2 = 0.5 * np.random.rand(output_number, hidden_unit_number) - 0.1
# 2*1
b2 = 0.5 * np.random.rand(output_number, 1) - 0.1


In [17]:
# 设置函数

def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))


In [23]:
mse_history = []
for i in range(max_epochs):
    # fp
    # 计算隐藏层
    hidden_out = sigmoid(np.dot(w1, sample_in) + b1)
    network_out = np.dot(w2, hidden_out) + b2
    # 误差计算
    err = sample_out - network_out
    mse = np.average(np.square(err))
    mse_history.append(mse)
    if mse < mse_final:
        break

    # bp
    delta2 = -err
    delta1 = np.dot(w2.transpose(), delta2) * hidden_out * (1 - hidden_out)

    # w2的偏导
    delta_w2 = np.dot(delta2, hidden_out.transpose())
    delta_b2 = np.dot(delta2, np.ones((sample_number, 1)))

    delta_w1 = np.dot(delta1, sample_in.transpose())
    delta_b1 = np.dot(delta1, np.ones((sample_number, 1)))

    w2 -= learn_rate * delta_w2
    w1 -= learn_rate * delta_w1
    b2 -= learn_rate * delta_b2
    b1 -= learn_rate * delta_b1


C:\Users\LeonardoZhou\AppData\Local\Temp\ipykernel_36812\1578747997.py:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-z))
c:\Users\LeonardoZhou\PycharmProjects\长三角数学建模\venv\lib\site-packages\numpy\core\_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\LeonardoZhou\AppData\Local\Temp\ipykernel_36812\82849913.py:9: RuntimeWarning: overflow encountered in square
  mse = np.average(np.square(err))
C:\Users\LeonardoZhou\AppData\Local\Temp\ipykernel_36812\82849913.py:16: RuntimeWarning: invalid value encountered in multiply
  delta1 = np.dot(w2.transpose(), delta2) * hidden_out * (1 - hidden_out)
